In [27]:
from os import path as osp

#import argparse
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, GAE, VGAE
from torch_geometric.utils import train_test_split_edges

import numpy as np

In [ ]:
#parser = argparse.ArgumentParser()
parser.add_argument('--variational', action='store_true')
parser.add_argument('--linear', action='store_true')
parser.add_argument('--dataset', type=str, default='Cora',
                    choices=['Cora', 'CiteSeer', 'PubMed'])
parser.add_argument('--epochs', type=int, default=400)
args = parser.parse_args()

In [5]:
path = osp.join(osp.dirname(osp.realpath('./')), '..', 'data', 'Planetoid')
path

'/home/abiricz/Projects/../data/Planetoid'

In [7]:
dataset = Planetoid(path, 'Cora', transform=T.NormalizeFeatures())


Processing...
Done!


In [29]:
dataset = Planetoid(path, 'CiteSeer', transform=T.NormalizeFeatures())
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = data.y = None
data = train_test_split_edges(data)

Processing...
Done!


In [30]:
data

Data(test_neg_edge_index=[2, 455], test_pos_edge_index=[2, 455], train_neg_adj_mask=[3327, 3327], train_pos_edge_index=[2, 7740], val_neg_edge_index=[2, 227], val_pos_edge_index=[2, 227], x=[3327, 3703])

In [31]:
data.train_pos_edge_index

tensor([[   0,    1,    1,  ..., 3324, 3325, 3326],
        [ 628,  158,  486,  ..., 2820, 1643,   33]])

In [33]:
dataset.num_classes, dataset.num_edge_features, dataset.num_node_features

(6, 0, 3703)

In [16]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True)
        self.conv2 = GCNConv(2 * out_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)


class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VariationalGCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True)
        self.conv_mu = GCNConv(2 * out_channels, out_channels, cached=True)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)


class LinearEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(LinearEncoder, self).__init__()
        self.conv = GCNConv(in_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        return self.conv(x, edge_index)


class VariationalLinearEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VariationalLinearEncoder, self).__init__()
        self.conv_mu = GCNConv(in_channels, out_channels, cached=True)
        self.conv_logstd = GCNConv(in_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

In [17]:
out_channels = 16
num_features = dataset.num_features

In [18]:
model = GAE( GCNEncoder(num_features, out_channels) )

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [20]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    #if args.variational:
    #    loss = loss + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)

In [21]:
def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [22]:
for epoch in range(1, 400+1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

Epoch: 001, AUC: 0.6588, AP: 0.6928
Epoch: 002, AUC: 0.6511, AP: 0.6858
Epoch: 003, AUC: 0.6493, AP: 0.6837
Epoch: 004, AUC: 0.6502, AP: 0.6848
Epoch: 005, AUC: 0.6531, AP: 0.6887
Epoch: 006, AUC: 0.6564, AP: 0.6923
Epoch: 007, AUC: 0.6594, AP: 0.6973
Epoch: 008, AUC: 0.6612, AP: 0.7005
Epoch: 009, AUC: 0.6628, AP: 0.7035
Epoch: 010, AUC: 0.6633, AP: 0.7049
Epoch: 011, AUC: 0.6639, AP: 0.7063
Epoch: 012, AUC: 0.6651, AP: 0.7087
Epoch: 013, AUC: 0.6674, AP: 0.7127
Epoch: 014, AUC: 0.6728, AP: 0.7182
Epoch: 015, AUC: 0.6813, AP: 0.7250
Epoch: 016, AUC: 0.6880, AP: 0.7295
Epoch: 017, AUC: 0.6926, AP: 0.7328
Epoch: 018, AUC: 0.6964, AP: 0.7357
Epoch: 019, AUC: 0.7041, AP: 0.7409
Epoch: 020, AUC: 0.7196, AP: 0.7509
Epoch: 021, AUC: 0.7414, AP: 0.7650
Epoch: 022, AUC: 0.7576, AP: 0.7771
Epoch: 023, AUC: 0.7665, AP: 0.7840
Epoch: 024, AUC: 0.7712, AP: 0.7875
Epoch: 025, AUC: 0.7783, AP: 0.7927
Epoch: 026, AUC: 0.7900, AP: 0.8016
Epoch: 027, AUC: 0.8008, AP: 0.8093
Epoch: 028, AUC: 0.8054, AP:

Epoch: 235, AUC: 0.9024, AP: 0.8979
Epoch: 236, AUC: 0.9022, AP: 0.8975
Epoch: 237, AUC: 0.9020, AP: 0.8976
Epoch: 238, AUC: 0.9017, AP: 0.8972
Epoch: 239, AUC: 0.9016, AP: 0.8970
Epoch: 240, AUC: 0.9017, AP: 0.8969
Epoch: 241, AUC: 0.9022, AP: 0.8974
Epoch: 242, AUC: 0.9024, AP: 0.8979
Epoch: 243, AUC: 0.9024, AP: 0.8979
Epoch: 244, AUC: 0.9020, AP: 0.8973
Epoch: 245, AUC: 0.9018, AP: 0.8966
Epoch: 246, AUC: 0.9016, AP: 0.8965
Epoch: 247, AUC: 0.9022, AP: 0.8974
Epoch: 248, AUC: 0.9021, AP: 0.8974
Epoch: 249, AUC: 0.9019, AP: 0.8970
Epoch: 250, AUC: 0.9015, AP: 0.8965
Epoch: 251, AUC: 0.9013, AP: 0.8961
Epoch: 252, AUC: 0.9012, AP: 0.8963
Epoch: 253, AUC: 0.9002, AP: 0.8956
Epoch: 254, AUC: 0.8993, AP: 0.8946
Epoch: 255, AUC: 0.8991, AP: 0.8945
Epoch: 256, AUC: 0.8997, AP: 0.8953
Epoch: 257, AUC: 0.9001, AP: 0.8956
Epoch: 258, AUC: 0.8996, AP: 0.8952
Epoch: 259, AUC: 0.8986, AP: 0.8945
Epoch: 260, AUC: 0.8979, AP: 0.8935
Epoch: 261, AUC: 0.8986, AP: 0.8940
Epoch: 262, AUC: 0.8995, AP: